In [14]:
import os
os.environ["NAPARI_ASYNC"] = "1"
from laptrack import datasets, LapTrack
import napari
import pandas as pd
import tracksdata as td
image, labels = datasets.cell_segmentation()
viewer = napari.Viewer()
viewer.add_image(image, name='image')
viewer.add_labels(labels, name='labels')

<Labels layer 'labels' at 0x32ff90290>

In [2]:
node_op = td.nodes.RegionPropsNodes(
    extra_properties=["label", "centroid", "area","intensity_mean"]
)
graph = td.graph.RustWorkXGraph()
node_op.add_nodes(graph, labels=labels, intensity_image=image)

Adding region properties nodes: 100%|██████████| 10/10 [00:00<00:00, 603.82it/s]


In [3]:
nodes_df = graph.node_attrs().to_pandas()
# Expand centroid tuple into separate columns
centroid_df = pd.DataFrame(nodes_df['centroid'].tolist(), columns=['centroid-y', 'centroid-x'])
nodes_df = pd.concat([nodes_df.drop(columns=['centroid']), centroid_df], axis=1)
nodes_df.head()

,node_id,t,mask,bbox,label,y,x,area,intensity_mean,centroid-y,centroid-x
0,0,0,"Mask(bbox=[131:144, 91:103])","[131, 91, 144, 103]",1,137.033613,96.336134,119.0,72.705882,137.033613,96.336134
1,1,0,"Mask(bbox=[140:151, 175:184])","[140, 175, 151, 184]",2,145.068966,178.988506,87.0,107.229885,145.068966,178.988506
2,2,0,"Mask(bbox=[118:131, 198:211])","[118, 198, 131, 211]",3,123.091743,203.266055,109.0,94.944954,123.091743,203.266055
3,3,0,"Mask(bbox=[57:70, 133:149])","[57, 133, 70, 149]",4,62.716216,141.290541,148.0,77.932432,62.716216,141.290541
4,4,0,"Mask(bbox=[29:39, 119:130])","[29, 119, 39, 130]",5,32.976744,124.348837,86.0,64.069767,32.976744,124.348837


In [4]:
lt = LapTrack(
    metric="sqeuclidean",
    cutoff=15**2,
    splitting_metric="sqeuclidean",
    splitting_cutoff=15**2
)
track_df, split_df, _ = lt.predict_dataframe(nodes_df, 
                                             frame_col='t', 
                                             coordinate_cols=['centroid-y', 'centroid-x'],
                                             only_coordinate_cols=False)
display(track_df.head())
display(split_df.head())

node_id  t                           mask                  bbox  \
frame index                                                                    
0     0            0  0   Mask(bbox=[131:144, 91:103])   [131, 91, 144, 103]   
      1            1  0  Mask(bbox=[140:151, 175:184])  [140, 175, 151, 184]   
      2            2  0  Mask(bbox=[118:131, 198:211])  [118, 198, 131, 211]   
      3            3  0    Mask(bbox=[57:70, 133:149])    [57, 133, 70, 149]   
      4            4  0    Mask(bbox=[29:39, 119:130])    [29, 119, 39, 130]   

             label           y           x   area  intensity_mean  centroid-y  \
frame index                                                                     
0     0          1  137.033613   96.336134  119.0       72.705882  137.033613   
      1          2  145.068966  178.988506   87.0      107.229885  145.068966   
      2          3  123.091743  203.266055  109.0       94.944954  123.091743   
      3          4   62.716216  141.290541  148.0       77.932432   62.716216   
      4          5   32.976744  124.348837   86.0       64.069767   32.976744   

             centroid-x  tree_id  track_id  
frame index                                 
0     0       96.336134        0         0  
      1      178.988506        1         1  
      2      203.266055        2         2  
      3      141.290541        3         3  
      4      124.348837        4         4

,parent_track_id,child_track_id
0,6,9
1,6,8
2,7,10
3,7,11


In [5]:
track_df2 = track_df.reset_index().sort_values("frame")
# Updating the graph with track IDs
if "track_id" not in graph.node_attr_keys:
    graph.add_node_attr_key("track_id", 0)
graph.update_node_attrs(
    attrs = {"track_id":track_df["track_id"].to_list()},
    node_ids=track_df["node_id"].to_list(),
)
for _, grp in track_df2.groupby("track_id"):
    node_ids = grp["node_id"].to_list()
    graph.bulk_add_edges([
        {"source_id":node_ids[i], "target_id":node_ids[i+1]} for i in range(len(node_ids)-1)
    ])

# Adding splitting edges
first_node_df = track_df2.drop_duplicates(subset=["track_id"], keep="first")
last_node_df = track_df2.drop_duplicates(subset=["track_id"], keep="last")
split_df2 = split_df.merge(
    first_node_df[["track_id", "node_id"]].rename(columns={"node_id":"child_node_id"}),
    left_on="child_track_id",
    right_on="track_id",
    how="left"
).merge(
    last_node_df[["track_id", "node_id"]].rename(columns={"node_id":"parent_node_id"}),
    left_on="parent_track_id",
    right_on="track_id",
    how="left"
)[["parent_node_id", "child_node_id"]]
graph.bulk_add_edges(
    [{"source_id":row["parent_node_id"], "target_id":row["child_node_id"]} for _, row in split_df2.iterrows()]
)

In [6]:
graph.node_attrs().head()

node_id,t,mask,bbox,label,y,x,centroid,area,intensity_mean,track_id
i64,i64,object,"array[i64, 4]",i64,f64,f64,"array[f64, 2]",f64,f64,i64
0,0,"Mask(bbox=[131:144, 91:103])","[131, 91, … 103]",1,137.033613,96.336134,"[137.033613, 96.336134]",119.0,72.705882,0
1,0,"Mask(bbox=[140:151, 175:184])","[140, 175, … 184]",2,145.068966,178.988506,"[145.068966, 178.988506]",87.0,107.229885,1
2,0,"Mask(bbox=[118:131, 198:211])","[118, 198, … 211]",3,123.091743,203.266055,"[123.091743, 203.266055]",109.0,94.944954,2
3,0,"Mask(bbox=[57:70, 133:149])","[57, 133, … 149]",4,62.716216,141.290541,"[62.716216, 141.290541]",148.0,77.932432,3
4,0,"Mask(bbox=[29:39, 119:130])","[29, 119, … 130]",5,32.976744,124.348837,"[32.976744, 124.348837]",86.0,64.069767,4


In [7]:
graph_view = td.array.GraphArrayView(graph, labels.shape, attr_key="track_id")

In [ ]:
layer = viewer.add_labels(graph_view, name='graph_view')
#selected_layer = viewer.add_labels(np.zeros_like(labels), name='selected_nodes')

In [16]:
labels2 = labels.copy()
ll_ms=viewer.add_labels(
    [labels2, labels2[::2,::2]], name='labels downsampled'
)

In [19]:
labels2 = np.tile(labels[:, :, :], (1,20,20))

Traceback (most recent call last):
  File "/Users/fukai/projects/napari-travali2/.venv/lib/python3.12/site-packages/napari/utils/events/event.py", line 777, in _invoke_callback
    cb(event)
  File "/Users/fukai/projects/napari-travali2/.venv/lib/python3.12/site-packages/napari/_qt/layer_controls/qt_layer_controls_container.py", line 128, in _display
    self.setCurrentWidget(self.empty_widget)
RuntimeError: wrapped C/C++ object of type QtLayerControlsContainer has been deleted

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/fukai/projects/napari-travali2/.venv/lib/python3.12/site-packages/napari/_qt/qt_main_window.py", line 658, in closeEvent
    quit_app_()
  File "/Users/fukai/projects/napari-travali2/.venv/lib/python3.12/site-packages/napari/_qt/qt_event_loop.py", line 261, in quit_app
    v.close()
  File "/Users/fukai/projects/napari-travali2/.venv/lib/python3.12/site-packages/napari/viewer.py", line 270, in 

In [17]:
labels2[:,:100,:] = 10

In [18]:
ll_ms.refresh()

In [10]:
layer_ms = viewer.add_labels(
    [graph_view, graph_view[::2,::2]]
)

AttributeError: 'GraphArrayView' object has no attribute 'size'

In [37]:
bbox_spatial_filter = graph.bbox_spatial_filter(
    frame_attr_key="t",
    bbox_attr_key="bbox"
)

In [38]:
graph.node_attr_keys

['node_id',
 't',
 'mask',
 'bbox',
 'y',
 'x',
 'label',
 'centroid',
 'area',
 'intensity_mean',
 'track_id']

In [39]:
from os import path
LOGGING_PATH = ".napari-travali/log.txt"
import logging 
import numpy as np

logging_path = path.join(path.expanduser("~"), LOGGING_PATH)
os.makedirs(path.dirname(logging_path), exist_ok=True)

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)  # Set the logger level to DEBUG
file_handler = logging.FileHandler(logging_path)
file_handler.setLevel(logging.DEBUG)  # Set the handler level to DEBUG
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)
logger.propagate = False  # Prevent propagation to the root logger
logger.info("Track clicked")

def track_clicked(viewer, event):
    logger.info(event.modifiers)
    logger.info("Track clicked")
    yield  # important to avoid a potential bug when selecting the daughter
    logger.info("button released")
    data_coordinates = layer.world_to_data(event.position)
    logger.debug(f"world coordinates: {event.position}")
    logger.debug(f"data coordinates: {data_coordinates}")
    nodes_df = bbox_spatial_filter[tuple([
        slice(c,c) for c in data_coordinates
    ])].node_attrs(attr_keys=["node_id", "track_id"])
    if nodes_df.is_empty():
        logger.info("No nodes in the bbox.")
        selected_layer.data = np.zeros_like(labels)
    track_id = nodes_df["track_id"].to_list()[0]
    sorted_node_ids = graph.filter(td.NodeAttr("track_id") == track_id).node_attrs(["node_id","t"]).sort("t")["node_id"]
    last_node_id = sorted_node_ids.last()
    successors = graph.successors(last_node_id, attr_keys=["track_id"])
    if len(successors) == 0:
        logger.info("No successors found.")
        successor_track_ids = []
    else:
        successor_track_ids = successors["track_id"].to_list()
    # get the row of nodes_df (polars dataframe) with the selected track_id and largest t
    #first_node_id = sorted_node_ids.first()
    logger.info(f"nodes in the bbox: {nodes_df["node_id"].to_list()}, "
                f"selected track_id: {track_id}, "
                f"successor track_ids: {successor_track_ids}.")
    selected_graph = graph.filter(td.NodeAttr("track_id").is_in([track_id, *successor_track_ids])).subgraph()
    graph_view = td.array.GraphArrayView(selected_graph, labels.shape, attr_key="track_id")
    selected_layer.data = graph_view
    
    
viewer.mouse_drag_callbacks.clear()
viewer.mouse_drag_callbacks.append(track_clicked)

In [ ]:
from napari import Viewer
from napari_travali2._stateful_widget import StateMachineWidget
from napari_travali2._logging import logger
import numpy as np

logger.setLevel("DEBUG")
logger.info("Starting napari-travali2")

viewer = Viewer()
widget = StateMachineWidget(viewer, ta, image, 
                            verified_track_ids=ta.attrs.get("verified_track_ids", []),
                            candidate_track_ids=ta.attrs.get("candidate_track_ids", []),
                            crop_size=100)
viewer.window.add_dock_widget(widget, area="right")
viewer.dims.set_current_step(0,0)

INFO:napari_travali2._logging:Region clicked
INFO:napari_travali2._logging:clicked at [  0 143 180]
INFO:napari_travali2._logging:Region selected: coords [  0 143 180]
INFO:napari_travali2._logging:window selected: (slice(None, None, None), slice(np.int64(93), np.int64(193), None), slice(np.int64(130), np.int64(230), None))
INFO:napari_travali2._logging:()
INFO:napari_travali2._logging:Track clicked
INFO:napari_travali2._logging:button released
DEBUG:napari_travali2._logging:world coordinates: (np.float64(3.0), np.float64(144.66757555054843), np.float64(180.98630938502362))
DEBUG:napari_travali2._logging:data coordinates: [ 3.         51.66757555 50.98630939]
INFO:napari_travali2._logging:clicked at [ 3 52 51] at frame 3 and label value 2
INFO:napari_travali2._logging:Track selected: frame 3 value 2
INFO:napari_travali2._logging:()
INFO:napari_travali2._logging:Track clicked
INFO:napari_travali2._logging:button released
DEBUG:napari_travali2._logging:world coordinates: (np.float64(5.0)